# AutoKaggle: Phase-Based Multi-Agent System

## Mục Tiêu

Notebook này tập trung vào phân tích sâu hệ thống đa tác tử dựa trên giai đoạn (phase-based multi-agent system) được giới thiệu trong bài báo "AutoKaggle: A Multi-Agent Framework for Autonomous Data Science Competitions". Chúng ta sẽ khám phá cách thiết kế và triển khai một hệ thống đa tác tử phối hợp nhau để xử lý các nhiệm vụ khoa học dữ liệu phức tạp theo từng giai đoạn.

Sau khi hoàn thành notebook này, bạn sẽ:
1. Hiểu được cách thức phân chia quy trình khoa học dữ liệu thành các giai đoạn rõ ràng
2. Nắm được vai trò và trách nhiệm của từng tác tử trong hệ thống
3. Hiểu được cách các tác tử phối hợp với nhau để giải quyết vấn đề
4. Có thể triển khai một hệ thống đa tác tử đơn giản sử dụng LangChain và LangGraph

## Trích Xuất Từ Bài Báo

> "The core concept of AutoKaggle is phase-based multi-agent reasoning. This method leverages LLMs to reason and solve tasks within a structured workflow, addressing different facets of the data science process through the collaboration of multiple agents. AutoKaggle comprises two main components: a phase-based workflow and a multi-agent system, which complement each other, as shown in Figure 1." (Section 3.1 Overall Framework, Page 4)

> "Phase-based Workflow. The data science process is divided into six key phases: understanding the background, preliminary exploratory data analysis, data cleaning, in-depth exploratory data analysis, feature engineering, and model building, validation, and prediction. Data cleaning, feature engineering, and model building, validation, and prediction are fundamental processes required for any data science competition. We designed two additional data analysis phases to provide essential information and insights for data cleaning and feature engineering, respectively." (Section 3.1 Overall Framework, Page 4)

> "Multi-agent System. The system consists of five specialized agents: Reader, Planner, Developer, Reviewer, and Summarizer. Each agent is designed to perform specific tasks within the workflow. They collaborate to analyze the problem, develop strategies, implement solutions, evaluate results, and generate comprehensive reports." (Section 3.1 Overall Framework, Page 4)

![AutoKaggle Framework](https://m-a-p.ai/AutoKaggle.github.io/assets/images/fig1.png)
*(Note: Diagram representation - actual image not included)*

## Cài Đặt Môi Trường

In [ ]:
!pip install -q langchain langchain-openai langchain-community langchain-core
!pip install -q langgraph pandas numpy matplotlib seaborn

In [ ]:
import os
import json
import pandas as pd
import numpy as np
from typing import Dict, List, Optional, Tuple, Any, Union
from enum import Enum

# Import LangChain components
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage
from langchain_core.language_models import BaseChatModel
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser, JsonOutputParser

# Import LangGraph components
from langgraph.graph import StateGraph, END

# Set up environment variables if using OpenAI's API
os.environ["OPENAI_API_KEY"] = "YOUR_OPENAI_API_KEY"  # Replace with your actual key

## Phần 1: Phân Tích Quy Trình Dựa Trên Giai Đoạn (Phase-Based Workflow)

Bài báo AutoKaggle chia quy trình khoa học dữ liệu thành 6 giai đoạn chính. Mỗi giai đoạn có một mục tiêu cụ thể và các đầu vào/đầu ra riêng. Hãy định nghĩa các giai đoạn này và mục đích của chúng:

In [ ]:
class Phase(Enum):
    BACKGROUND_UNDERSTANDING = "Background Understanding"
    PRELIMINARY_EDA = "Preliminary EDA"
    DATA_CLEANING = "Data Cleaning"
    INDEPTH_EDA = "In-depth EDA"
    FEATURE_ENGINEERING = "Feature Engineering"
    MODEL_BUILDING = "Model Building, Validation and Prediction"
    COMPLETED = "Completed"

### Giải Thích Lý Thuyết Về Các Giai Đoạn

1. **Hiểu biết nền tảng (Background Understanding)**
   - **Mục tiêu**: Phân tích bối cảnh cuộc thi, hiểu cấu trúc dữ liệu, định nghĩa vấn đề và xác định biến mục tiêu
   - **Input**: Thông tin tổng quan cuộc thi, mô tả dữ liệu, tệp dữ liệu gốc
   - **Output**: Báo cáo cấu trúc cuộc thi, mục tiêu, định nghĩa vấn đề và tổng quan dữ liệu
   - **Lý do quan trọng**: Thiết lập nền tảng cho toàn bộ quy trình, đảm bảo hiểu chính xác yêu cầu của cuộc thi

2. **Phân tích dữ liệu khám phá sơ bộ (Preliminary EDA)**
   - **Mục tiêu**: Khám phá cấu trúc dữ liệu, phát hiện vấn đề cần xử lý, hiểu phân phối và mối quan hệ giữa các biến
   - **Input**: Dữ liệu gốc, báo cáo từ giai đoạn hiểu biết nền tảng
   - **Output**: Phân tích cơ bản về dữ liệu, danh sách vấn đề cần xử lý (dữ liệu thiếu, dữ liệu ngoại lai, v.v.)
   - **Lý do quan trọng**: Cung cấp thông tin cần thiết để làm sạch dữ liệu một cách có chiến lược

3. **Làm sạch dữ liệu (Data Cleaning)**
   - **Mục tiêu**: Xử lý dữ liệu thiếu, dữ liệu ngoại lai, chuyển đổi kiểu dữ liệu, chuẩn hóa dữ liệu
   - **Input**: Dữ liệu gốc, báo cáo từ phân tích dữ liệu sơ bộ
   - **Output**: Dữ liệu đã được làm sạch, báo cáo về quá trình làm sạch
   - **Lý do quan trọng**: Đảm bảo dữ liệu có chất lượng cao cho các giai đoạn tiếp theo

4. **Phân tích dữ liệu khám phá chuyên sâu (In-depth EDA)**
   - **Mục tiêu**: Phân tích sâu hơn về dữ liệu đã làm sạch, hiểu rõ hơn về mối quan hệ giữa các biến, đặc biệt là với biến mục tiêu
   - **Input**: Dữ liệu đã làm sạch
   - **Output**: Báo cáo phân tích sâu, các phát hiện quan trọng, định hướng cho việc tạo đặc trưng
   - **Lý do quan trọng**: Định hướng chiến lược cho việc kỹ thuật đặc trưng

5. **Kỹ thuật đặc trưng (Feature Engineering)**
   - **Mục tiêu**: Tạo, chọn và biến đổi đặc trưng để tăng hiệu suất mô hình
   - **Input**: Dữ liệu đã làm sạch, báo cáo từ phân tích dữ liệu chuyên sâu
   - **Output**: Dữ liệu với các đặc trưng mới, báo cáo về quá trình kỹ thuật đặc trưng
   - **Lý do quan trọng**: Nâng cao khả năng dự đoán của mô hình thông qua đặc trưng chất lượng cao

6. **Xây dựng, xác thực và dự đoán mô hình (Model Building, Validation and Prediction)**
   - **Mục tiêu**: Xây dựng, đánh giá mô hình và tạo dự đoán cho tập kiểm thử
   - **Input**: Dữ liệu đã qua kỹ thuật đặc trưng
   - **Output**: Mô hình tốt nhất, dự đoán cuối cùng, file submission.csv
   - **Lý do quan trọng**: Hoàn thiện quy trình khoa học dữ liệu và tạo kết quả cuối cùng

Việc phân chia quy trình khoa học dữ liệu thành các giai đoạn rõ ràng có những lợi ích sau:
1. **Tách biệt các mối quan tâm**: Mỗi giai đoạn tập trung vào một khía cạnh cụ thể của quy trình
2. **Kiểm soát chất lượng**: Có thể kiểm tra kết quả của mỗi giai đoạn trước khi chuyển sang giai đoạn tiếp theo
3. **Quản lý phức tạp**: Chia các nhiệm vụ phức tạp thành các phần nhỏ hơn, dễ quản lý hơn
4. **Khắc phục lỗi**: Lỗi ở mỗi giai đoạn có thể được khắc phục mà không ảnh hưởng đến toàn bộ quy trình

## Phần 2: Hệ Thống Đa Tác Tử (Multi-Agent System)

AutoKaggle sử dụng 5 tác tử chuyên biệt, mỗi tác tử đảm nhiệm một vai trò cụ thể trong quy trình. Hãy triển khai các tác tử này:

In [ ]:
# Định nghĩa trạng thái (state) của hệ thống
class AutoKaggleState:
    def __init__(self):
        # Thông tin về giai đoạn hiện tại
        self.current_phase = Phase.BACKGROUND_UNDERSTANDING
        
        # Thông tin và dữ liệu của cuộc thi
        self.competition_info = {}
        self.data = {}
        
        # Kết quả từ các giai đoạn
        self.reports = {}
        self.plans = {}
        self.code = {}
        self.execution_results = {}
        
    def update_phase(self, phase: Phase):
        self.current_phase = phase
        
    def to_dict(self):
        return {
            "current_phase": self.current_phase.value,
            "competition_info": self.competition_info,
            "reports": self.reports,
            "plans": self.plans,
            "code": self.code,
            "execution_results": self.execution_results
        }

In [ ]:
# Tác tử 1: Reader
class ReaderAgent:
    def __init__(self, model: BaseChatModel):
        """
        Khởi tạo Reader Agent với mô hình ngôn ngữ lớn
        
        Parameters:
        - model: Mô hình ngôn ngữ lớn dùng cho tác tử
        """
        self.model = model
        self.system_prompt = """
        Bạn là một Reader Agent chuyên đọc và phân tích thông tin cuộc thi Kaggle.
        
        Nhiệm vụ của bạn là đọc và tóm tắt thông tin cuộc thi thành một báo cáo có cấu trúc rõ ràng, bao gồm:
        1. Tổng quan cuộc thi (Competition Overview)
        2. Các tệp dữ liệu và mục đích của chúng (Files and their purpose)
        3. Định nghĩa vấn đề (Problem Definition)
        4. Thông tin dữ liệu (Data Information)
           - Kiểu dữ liệu: ID, Numerical, Categorical, Datetime
           - Mô tả chi tiết cho mỗi cột
        5. Biến mục tiêu (Target Variable)
        6. Metrics đánh giá (Evaluation Metrics)
        7. Định dạng nộp bài (Submission Format)
        8. Các khía cạnh quan trọng khác (Other Key Aspects)
        
        Đảm bảo phân tích đầy đủ, tập trung vào việc hiểu rõ mục đích của từng tệp, biến mục tiêu, và metrics đánh giá.
        """
        
        self.prompt = ChatPromptTemplate.from_messages([
            SystemMessage(content=self.system_prompt),
            MessagesPlaceholder(variable_name="history"),
            HumanMessage(content="{input}")
        ])
        
        self.chain = self.prompt | self.model | StrOutputParser()
    
    def execute(self, competition_overview: str, data_samples: Dict[str, pd.DataFrame]) -> str:
        """
        Đọc và phân tích thông tin cuộc thi và dữ liệu mẫu
        
        Parameters:
        - competition_overview: Nội dung tổng quan về cuộc thi
        - data_samples: Dictionary chứa các mẫu dữ liệu (train, test, etc.)
        
        Returns:
        - Báo cáo phân tích cuộc thi
        """
        # Chuẩn bị mô tả dữ liệu mẫu
        data_description = ""
        for name, df in data_samples.items():
            data_description += f"\n### {name} Sample:\n"
            data_description += f"Shape: {df.shape}\n"
            data_description += f"Columns: {list(df.columns)}\n"
            data_description += f"Data types:\n{df.dtypes}\n"
            data_description += f"First 5 rows:\n{df.head().to_string()}\n"
        
        # Tạo input cho LLM
        input_text = f"""# Competition Overview
        {competition_overview}
        
        # Data Samples
        {data_description}
        
        Please analyze this competition and provide a comprehensive report following the structure in your instructions.
        """
        
        # Thực thi chain
        result = self.chain.invoke({"input": input_text, "history": []})
        return result

In [ ]:
# Tác tử 2: Planner
class PlannerAgent:
    def __init__(self, model: BaseChatModel):
        """
        Khởi tạo Planner Agent với mô hình ngôn ngữ lớn
        
        Parameters:
        - model: Mô hình ngôn ngữ lớn dùng cho tác tử
        """
        self.model = model
        self.system_prompt = """
        Bạn là một Planner Agent chuyên tạo kế hoạch chi tiết cho các nhiệm vụ khoa học dữ liệu.
        
        Nhiệm vụ của bạn là phân tích giai đoạn hiện tại và tạo một kế hoạch chi tiết. Kế hoạch của bạn phải:
        1. Chia nhỏ giai đoạn thành tối đa 4 nhiệm vụ chính
        2. Với mỗi nhiệm vụ, cung cấp:
           - Mục tiêu rõ ràng
           - Phương pháp và cách tiếp cận cần thiết
           - Kết quả đầu ra mong đợi
           - Ràng buộc và các cân nhắc
        
        Tập trung chỉ vào giai đoạn hiện tại và đưa ra chỉ dẫn cụ thể cho từng đặc trưng cần xử lý.
        
        HƯỚNG DẪN LẬP KẾ HOẠCH:
        1. Giới hạn kế hoạch tối đa 4 nhiệm vụ
        2. Cung cấp phương pháp và ràng buộc rõ ràng cho mỗi nhiệm vụ
        3. Tập trung vào các bước quan trọng cụ thể cho giai đoạn hiện tại
        4. Ưu tiên các phương pháp và giá trị được đề cập trong yêu cầu
        5. Đưa ra kế hoạch chi tiết mà không viết code thực tế
        """
        
        self.prompt = ChatPromptTemplate.from_messages([
            SystemMessage(content=self.system_prompt),
            MessagesPlaceholder(variable_name="history"),
            HumanMessage(content="{input}")
        ])
        
        self.chain = self.prompt | self.model | StrOutputParser()
    
    def execute(self, phase: Phase, state_info: Dict) -> str:
        """
        Tạo kế hoạch cho một giai đoạn cụ thể
        
        Parameters:
        - phase: Giai đoạn hiện tại
        - state_info: Thông tin trạng thái hiện tại của hệ thống
        
        Returns:
        - Kế hoạch chi tiết cho giai đoạn
        """
        # Lấy thông tin liên quan từ state
        phase_name = phase.value
        competition_info = state_info.get("competition_info", {})
        previous_reports = state_info.get("reports", {})
        
        # Tạo context cho planner
        context = f"""# Current Phase: {phase_name}
        
        # Competition Information
        {json.dumps(competition_info, indent=2)}
        
        # Previous Reports
        {json.dumps(previous_reports, indent=2)}
        
        Create a detailed plan for the {phase_name} phase. Focus only on this phase's objectives and tasks.
        """
        
        # Thực thi chain
        result = self.chain.invoke({"input": context, "history": []})
        return result

In [ ]:
# Tác tử 3: Developer
class DeveloperAgent:
    def __init__(self, model: BaseChatModel):
        """
        Khởi tạo Developer Agent với mô hình ngôn ngữ lớn
        
        Parameters:
        - model: Mô hình ngôn ngữ lớn dùng cho tác tử
        """
        self.model = model
        self.system_prompt = """
        Bạn là một Developer Agent chuyên triển khai code cho các nhiệm vụ khoa học dữ liệu.
        
        Nhiệm vụ của bạn là viết code Python để thực hiện kế hoạch đã được cung cấp. Code của bạn cần:
        1. Cấu trúc tốt, hiệu quả và chính xác
        2. Tuân thủ nghiêm ngặt kế hoạch và triển khai tất cả các nhiệm vụ đã chỉ định
        3. Bao gồm các chú thích chi tiết để giải thích các hoạt động phức tạp
        4. Có thể được thực thi trực tiếp
        5. Có khả năng xử lý lỗi và xác thực đầu vào
        
        Khi code có lỗi, bạn sẽ gỡ lỗi bằng cách:
        1. Phân tích thông báo lỗi
        2. Xác định nguyên nhân gốc rễ
        3. Sửa chữa code
        4. Kiểm tra lại để xác nhận sửa chữa
        """
        
        self.prompt = ChatPromptTemplate.from_messages([
            SystemMessage(content=self.system_prompt),
            MessagesPlaceholder(variable_name="history"),
            HumanMessage(content="{input}")
        ])
        
        self.chain = self.prompt | self.model | StrOutputParser()
    
    def execute(self, phase: Phase, plan: str, error_message: str = None, previous_code: str = None) -> str:
        """
        Viết hoặc sửa code dựa trên kế hoạch và thông báo lỗi (nếu có)
        
        Parameters:
        - phase: Giai đoạn hiện tại
        - plan: Kế hoạch chi tiết cho giai đoạn
        - error_message: Thông báo lỗi (nếu có)
        - previous_code: Code trước đó (nếu đang gỡ lỗi)
        
        Returns:
        - Code triển khai hoặc đã được sửa lỗi
        """
        phase_name = phase.value
        
        if error_message and previous_code:
            # Đang trong chế độ gỡ lỗi
            input_text = f"""# Debug Code for Phase: {phase_name}
            
            ## Previous Code
            ```python
            {previous_code}
            ```
            
            ## Error Message
            ```
            {error_message}
            ```
            
            Please debug the code above. Analyze the error message, identify the root cause, and provide a fixed version of the code.
            """
        else:
            # Đang viết code mới dựa trên kế hoạch
            input_text = f"""# Implement Code for Phase: {phase_name}
            
            ## Plan
            {plan}
            
            Please implement Python code to execute this plan. Ensure your code is well-structured, efficient, and includes appropriate error handling.
            """
        
        # Thực thi chain
        result = self.chain.invoke({"input": input_text, "history": []})
        
        # Trích xuất code từ kết quả (nếu có định dạng markdown)
        if "```python" in result and "```" in result:
            code_blocks = []
            in_code_block = False
            for line in result.split("\n"):
                if line.strip() == "```python" or line.strip() == "```python3":
                    in_code_block = True
                    continue
                elif line.strip() == "```" and in_code_block:
                    in_code_block = False
                    continue
                elif in_code_block:
                    code_blocks.append(line)
            
            if code_blocks:
                return "\n".join(code_blocks)
        
        return result

In [ ]:
# Tác tử 4: Reviewer
class ReviewerAgent:
    def __init__(self, model: BaseChatModel):
        """
        Khởi tạo Reviewer Agent với mô hình ngôn ngữ lớn
        
        Parameters:
        - model: Mô hình ngôn ngữ lớn dùng cho tác tử
        """
        self.model = model
        self.system_prompt = """
        Bạn là một Reviewer Agent chuyên đánh giá chất lượng và tính đúng đắn của code và kết quả.
        
        Nhiệm vụ của bạn là xem xét code và kết quả thực thi để đảm bảo chúng đáp ứng các yêu cầu và mục tiêu. 
        Đánh giá dựa trên:
        1. Tính đúng đắn và hiệu quả của code
        2. Sự phù hợp với kế hoạch ban đầu
        3. Chất lượng đầu ra và kết quả
        4. Tuân thủ các thực hành tốt nhất
        
        Cung cấp phản hồi chi tiết nêu bật điểm mạnh và các lĩnh vực cần cải thiện.
        """
        
        self.prompt = ChatPromptTemplate.from_messages([
            SystemMessage(content=self.system_prompt),
            MessagesPlaceholder(variable_name="history"),
            HumanMessage(content="{input}")
        ])
        
        self.chain = self.prompt | self.model | StrOutputParser()
    
    def execute(self, phase: Phase, plan: str, code: str, execution_result: str) -> str:
        """
        Đánh giá code và kết quả thực thi
        
        Parameters:
        - phase: Giai đoạn hiện tại
        - plan: Kế hoạch ban đầu
        - code: Code đã triển khai
        - execution_result: Kết quả thực thi code
        
        Returns:
        - Báo cáo đánh giá
        """
        phase_name = phase.value
        
        input_text = f"""# Review for Phase: {phase_name}
        
        ## Original Plan
        {plan}
        
        ## Implemented Code
        ```python
        {code}
        ```
        
        ## Execution Results
        ```
        {execution_result}
        ```
        
        Please review the code and results above. Evaluate whether the implementation fulfills the plan's objectives, follows best practices, and produces the expected outputs.
        """
        
        # Thực thi chain
        result = self.chain.invoke({"input": input_text, "history": []})
        return result

In [ ]:
# Tác tử 5: Summarizer
class SummarizerAgent:
    def __init__(self, model: BaseChatModel):
        """
        Khởi tạo Summarizer Agent với mô hình ngôn ngữ lớn
        
        Parameters:
        - model: Mô hình ngôn ngữ lớn dùng cho tác tử
        """
        self.model = model
        self.system_prompt = """
        Bạn là một Summarizer Agent chuyên tạo báo cáo tổng hợp.
        
        Nhiệm vụ của bạn là tóm tắt công việc đã hoàn thành trong một giai đoạn, phân tích các yếu tố sau:
        1. Phát hiện và hiểu biết chính
        2. Hành động đã thực hiện và lý do
        3. Tác động đến dự án tổng thể
        4. Thách thức gặp phải và cách giải quyết
        5. Khuyến nghị cho các giai đoạn tiếp theo
        
        Tạo một báo cáo rõ ràng, có cấu trúc tốt, nêu bật các khía cạnh và bài học quan trọng nhất.
        """
        
        self.prompt = ChatPromptTemplate.from_messages([
            SystemMessage(content=self.system_prompt),
            MessagesPlaceholder(variable_name="history"),
            HumanMessage(content="{input}")
        ])
        
        self.chain = self.prompt | self.model | StrOutputParser()
    
    def execute(self, phase: Phase, plan: str, code: str, execution_result: str, review: str) -> str:
        """
        Tạo báo cáo tổng hợp cho một giai đoạn
        
        Parameters:
        - phase: Giai đoạn hiện tại
        - plan: Kế hoạch ban đầu
        - code: Code đã triển khai
        - execution_result: Kết quả thực thi code
        - review: Báo cáo đánh giá từ Reviewer
        
        Returns:
        - Báo cáo tổng hợp
        """
        phase_name = phase.value
        
        input_text = f"""# Create Summary Report for Phase: {phase_name}
        
        ## Plan
        {plan}
        
        ## Code Implementation
        ```python
        {code}
        ```
        
        ## Execution Results
        ```
        {execution_result}
        ```
        
        ## Review
        {review}
        
        Please create a comprehensive summary report for this phase, highlighting key findings, actions taken, impacts, challenges, and recommendations for the next phase.
        """
        
        # Thực thi chain
        result = self.chain.invoke({"input": input_text, "history": []})
        return result

## Phần 3: Sơ Đồ Luồng Làm Việc Đa Tác Tử

Dưới đây là cách các tác tử phối hợp với nhau theo từng giai đoạn. Chúng ta sẽ sử dụng LangGraph để mô hình hóa luồng công việc:

In [ ]:
# Xây dựng luồng làm việc đa tác tử
class PhaseBasedMultiAgentSystem:
    def __init__(self, model_name: str = "gpt-4o"):
        """
        Khởi tạo hệ thống đa tác tử dựa trên giai đoạn
        
        Parameters:
        - model_name: Tên mô hình LLM sử dụng cho các tác tử
        """
        # Khởi tạo LLM
        self.model = ChatOpenAI(model=model_name)
        
        # Khởi tạo các tác tử
        self.reader = ReaderAgent(self.model)
        self.planner = PlannerAgent(self.model)
        self.developer = DeveloperAgent(self.model)
        self.reviewer = ReviewerAgent(self.model)
        self.summarizer = SummarizerAgent(self.model)
        
        # Khởi tạo state
        self.state = AutoKaggleState()
        
        # Xây dựng workflow graph
        self._build_graph()
    
    def _build_graph(self):
        """
        Xây dựng đồ thị luồng công việc sử dụng LangGraph
        """
        # Khởi tạo StateGraph
        workflow = StateGraph(AutoKaggleState)
        
        # Định nghĩa các nút cho từng giai đoạn
        workflow.add_node("background_understanding", self._execute_background_understanding)
        workflow.add_node("preliminary_eda", self._execute_preliminary_eda)
        workflow.add_node("data_cleaning", self._execute_data_cleaning)
        workflow.add_node("indepth_eda", self._execute_indepth_eda)
        workflow.add_node("feature_engineering", self._execute_feature_engineering)
        workflow.add_node("model_building", self._execute_model_building)
        
        # Định nghĩa các cạnh cho luồng công việc
        workflow.add_edge("background_understanding", "preliminary_eda")
        workflow.add_edge("preliminary_eda", "data_cleaning")
        workflow.add_edge("data_cleaning", "indepth_eda")
        workflow.add_edge("indepth_eda", "feature_engineering")
        workflow.add_edge("feature_engineering", "model_building")
        workflow.add_edge("model_building", END)
        
        # Compiler đồ thị
        self.workflow = workflow.compile()
    
    def _execute_background_understanding(self, state):
        """
        Thực thi giai đoạn Background Understanding
        
        Trong giai đoạn này:
        1. Reader Agent đọc thông tin cuộc thi
        2. Reader Agent tạo báo cáo tóm tắt
        3. Cập nhật trạng thái
        """
        # Lấy thông tin cuộc thi từ state
        competition_info = state.competition_info
        overview = competition_info.get("overview", "")
        data_samples = competition_info.get("data_samples", {})
        
        # Reader đọc và phân tích thông tin
        report = self.reader.execute(overview, data_samples)
        
        # Cập nhật state
        state.reports[Phase.BACKGROUND_UNDERSTANDING.value] = report
        state.update_phase(Phase.PRELIMINARY_EDA)
        
        return state
    
    def _execute_preliminary_eda(self, state):
        """
        Thực thi giai đoạn Preliminary EDA
        
        Trong giai đoạn này:
        1. Planner tạo kế hoạch cho EDA sơ bộ
        2. Developer viết code
        3. Code được thực thi (giả lập)
        4. Reviewer đánh giá kết quả
        5. Summarizer tạo báo cáo tổng hợp
        """
        # Planner tạo kế hoạch
        plan = self.planner.execute(
            Phase.PRELIMINARY_EDA, 
            {"competition_info": state.competition_info, "reports": state.reports}
        )
        state.plans[Phase.PRELIMINARY_EDA.value] = plan
        
        # Developer viết code
        code = self.developer.execute(Phase.PRELIMINARY_EDA, plan)
        state.code[Phase.PRELIMINARY_EDA.value] = code
        
        # Giả lập thực thi code
        execution_result = "[Simulation] Preliminary EDA executed successfully with basic statistics and visualizations."
        state.execution_results[Phase.PRELIMINARY_EDA.value] = execution_result
        
        # Reviewer đánh giá
        review = self.reviewer.execute(Phase.PRELIMINARY_EDA, plan, code, execution_result)
        
        # Summarizer tạo báo cáo
        summary = self.summarizer.execute(Phase.PRELIMINARY_EDA, plan, code, execution_result, review)
        state.reports[Phase.PRELIMINARY_EDA.value] = summary
        
        # Cập nhật trạng thái
        state.update_phase(Phase.DATA_CLEANING)
        return state
    
    # Tương tự cho các giai đoạn khác...
    def _execute_data_cleaning(self, state):
        # Tương tự như _execute_preliminary_eda
        state.update_phase(Phase.INDEPTH_EDA)
        return state
    
    def _execute_indepth_eda(self, state):
        # Tương tự như _execute_preliminary_eda
        state.update_phase(Phase.FEATURE_ENGINEERING)
        return state
    
    def _execute_feature_engineering(self, state):
        # Tương tự như _execute_preliminary_eda
        state.update_phase(Phase.MODEL_BUILDING)
        return state
    
    def _execute_model_building(self, state):
        # Tương tự như _execute_preliminary_eda
        state.update_phase(Phase.COMPLETED)
        return state
    
    def run(self, competition_info: Dict) -> AutoKaggleState:
        """
        Chạy toàn bộ quy trình tự động
        
        Parameters:
        - competition_info: Thông tin cuộc thi và dữ liệu
        
        Returns:
        - Trạng thái cuối cùng sau khi hoàn thành tất cả các giai đoạn
        """
        self.state.competition_info = competition_info
        final_state = self.workflow.invoke(self.state)
        return final_state

## Phần 4: Mô Phỏng Giao Tiếp Giữa Các Tác Tử

Để hiểu rõ hơn về cách các tác tử phối hợp với nhau, hãy xem xét một ví dụ mô phỏng về luồng thông tin và giao tiếp giữa các tác tử:

In [ ]:
# Mô phỏng giao tiếp giữa các tác tử trong quá trình xử lý một phase
def simulate_agent_communication():
    """
    Mô phỏng quá trình giao tiếp giữa các tác tử trong hệ thống AutoKaggle
    
    Returns:
    - Dictionary chứa mô phỏng các thông điệp được trao đổi
    """
    # Giả lập dữ liệu và thông tin cuộc thi
    competition_info = {
        "name": "Titanic: Machine Learning from Disaster",
        "overview": "The sinking of the Titanic is one of the most infamous shipwrecks in history. This competition asks you to predict which passengers survived the tragedy.",
        "data_samples": {
            "train": "[Simulated train dataframe with 891 rows and 12 columns including PassengerId, Survived, Pclass, Name, Sex, Age, etc.]",
            "test": "[Simulated test dataframe with 418 rows and 11 columns, similar to train but without Survived column]"
        }
    }
    
    # Mô phỏng giao tiếp cho giai đoạn Background Understanding
    communication_flow = {
        "phase": Phase.BACKGROUND_UNDERSTANDING.value,
        "steps": [
            {
                "agent": "System",
                "action": "Initialize workflow",
                "message": "Starting AutoKaggle workflow with Titanic competition data"
            },
            {
                "agent": "Reader",
                "action": "Process competition info",
                "message": "Reading competition overview and sample data"
            },
            {
                "agent": "Reader",
                "action": "Generate report",
                "message": "Created comprehensive analysis of competition requirements, data structure, and evaluation metrics"
            },
            {
                "agent": "System",
                "action": "Update state",
                "message": "Updated state with background understanding report and advancing to Preliminary EDA phase"
            }
        ]
    }
    
    # Mô phỏng giao tiếp cho giai đoạn Preliminary EDA
    preliminary_eda_flow = {
        "phase": Phase.PRELIMINARY_EDA.value,
        "steps": [
            {
                "agent": "Planner",
                "action": "Create plan",
                "message": "Analyzing background report and creating plan for preliminary EDA"
            },
            {
                "agent": "Planner",
                "action": "Output plan",
                "message": "Created plan with 4 key tasks: 1) Basic data statistics, 2) Missing value analysis, 3) Data distribution visualization, 4) Initial correlation analysis"
            },
            {
                "agent": "Developer",
                "action": "Implement code",
                "message": "Writing Python code to implement the Preliminary EDA plan"
            },
            {
                "agent": "System",
                "action": "Execute code",
                "message": "Executing Preliminary EDA code"
            },
            {
                "agent": "Reviewer",
                "action": "Review results",
                "message": "Reviewing code and execution results against the original plan"
            },
            {
                "agent": "Reviewer",
                "action": "Output review",
                "message": "Code implements all required tasks successfully. Visualizations are clear and informative. Missing value analysis is thorough. Suggestion: Consider adding more comments to explain the significance of findings."
            },
            {
                "agent": "Summarizer",
                "action": "Create summary",
                "message": "Analyzing all inputs from previous agents to create comprehensive phase summary"
            },
            {
                "agent": "Summarizer",
                "action": "Output summary",
                "message": "Created detailed summary highlighting: 1) 19.8% missing Age values, 2) Higher survival rates for females, 3) Class correlation with survival, 4) Recommendations for data cleaning strategies"
            },
            {
                "agent": "System",
                "action": "Update state",
                "message": "Updated state with Preliminary EDA results and advancing to Data Cleaning phase"
            }
        ]
    }
    
    # Kết hợp tất cả các luồng giao tiếp
    all_communication = {
        "competition": "Titanic: Machine Learning from Disaster",
        "workflow": [
            communication_flow,
            preliminary_eda_flow
            # Có thể thêm luồng cho các giai đoạn khác...
        ]
    }
    
    return all_communication

# Chạy mô phỏng (comment lại để không thực thi)
# communication_example = simulate_agent_communication()
# print(json.dumps(communication_example, indent=2))

## Phân Tích Lợi Ích Và Thách Thức

### Lợi Ích của Hệ Thống Đa Tác Tử Dựa Trên Giai Đoạn

1. **Phân Chia Trách Nhiệm Rõ Ràng**
   - Mỗi tác tử có một vai trò chuyên biệt, tập trung vào một nhiệm vụ cụ thể
   - Giúp giảm thiểu khả năng một tác tử phải xử lý quá nhiều nhiệm vụ khác nhau

2. **Mô-đun Hóa và Khả Năng Mở Rộng**
   - Dễ dàng thêm, xóa hoặc thay thế các tác tử mà không ảnh hưởng đến toàn bộ hệ thống
   - Có thể thay đổi triển khai của một tác tử mà không cần sửa đổi các tác tử khác

3. **Kiểm Soát Chất Lượng Tốt Hơn**
   - Mỗi giai đoạn có Reviewer đánh giá kết quả trước khi chuyển sang giai đoạn tiếp theo
   - Lỗi được phát hiện sớm và không lan truyền qua các giai đoạn

4. **Báo Cáo và Tài Liệu Đầy Đủ**
   - Summarizer tạo báo cáo sau mỗi giai đoạn, cung cấp tài liệu và giải thích cho quá trình ra quyết định
   - Tăng tính minh bạch và khả năng giải thích của hệ thống

### Thách Thức và Hướng Giải Quyết

1. **Phối Hợp và Giao Tiếp Giữa Các Tác Tử**
   - **Thách thức**: Đảm bảo thông tin được truyền đạt hiệu quả giữa các tác tử
   - **Giải pháp**: Sử dụng đối tượng state chia sẻ và định nghĩa rõ ràng các giao diện giữa các tác tử

2. **Xử Lý Lỗi và Phục Hồi**
   - **Thách thức**: Khi một tác tử gặp lỗi, toàn bộ luồng có thể bị gián đoạn
   - **Giải pháp**: Triển khai cơ chế xử lý lỗi mạnh mẽ và khả năng thử lại với chiến lược khác

3. **Độ Trễ và Hiệu Suất**
   - **Thách thức**: Mỗi tác tử yêu cầu một lần gọi LLM riêng biệt, có thể dẫn đến độ trễ cao
   - **Giải pháp**: Tối ưu hóa prompt, sử dụng bộ nhớ đệm, và thực hiện các tác vụ song song khi có thể

4. **Tính Nhất Quán Trong Ra Quyết Định**
   - **Thách thức**: Đảm bảo các quyết định và phương pháp tiếp cận nhất quán giữa các tác tử
   - **Giải pháp**: Thiết kế prompt kỹ lưỡng và duy trì context giữa các tác tử

## Ví Dụ Mã Giả Mô Tả Thuật Toán Chính

Thuật toán của hệ thống đa tác tử dựa trên giai đoạn trong AutoKaggle có thể được tóm tắt như sau:

```
ALGORITHM: Phase-Based Multi-Agent Workflow
INPUT: Competition C, Dataset D
OUTPUT: Solution S, Comprehensive Report R

1. Initialize state s0 with first phase φ1: "Understand Background"
2. t ← 0
3. Φ ← {φ1, φ2, ..., φ6} // Set of all phases
4. Define Aφ for each φ ∈ Φ // Agents for each phase
5. do
6.     st ← GetCurrentState()
7.     φcurrent ← GetCurrentPhase(Φ)
8.     At ← Aφcurrent
9.     for a ∈ At do
10.        if a is Planner then
11.            ra ← a.execute(st)
12.            st ← UpdateState(st, ra)
13.            if UserInteractionEnabled() then
14.                st ← UserReview(st) // User Review plan
15.        else if a is Developer then
16.            ra ← a.execute(st)
17.            st ← UpdateState(st, ra)
18.            if NoErrors(ra) then
19.                T ← ExecuteUnitTests(φcurrent)
20.                if ¬PassTests(T) then
21.                    st ← Debug(st)
22.        else
23.            ra ← a.execute(st)
24.            st ← UpdateState(st, ra)
25.        if AllAgentsCompleted(At) and PassTests(T) then
26.            φcurrent ← NextPhase(Φ)
27.            t ← t + 1
28. while ∃φ ∈ Φ : not completed(φ)
29. S ← ExtractSolution(st)
30. R ← GenerateReport(st)
31. return S, R
```

## Kiểm Tra và Trực Quan Hóa Hệ Thống

Chúng ta có thể trực quan hóa hệ thống đa tác tử dựa trên giai đoạn bằng một sơ đồ luồng công việc:

In [ ]:
# Code để vẽ sơ đồ luồng công việc của hệ thống đa tác tử
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from matplotlib.path import Path
import numpy as np

def visualize_multi_agent_system():
    # Tạo figure và axes
    fig, ax = plt.subplots(figsize=(14, 10))
    
    # Định nghĩa các giai đoạn và tác tử
    phases = [
        "Background\nUnderstanding",
        "Preliminary\nEDA",
        "Data\nCleaning",
        "In-depth\nEDA",
        "Feature\nEngineering",
        "Model Building\n& Prediction"
    ]
    
    agents = ["Reader", "Planner", "Developer", "Reviewer", "Summarizer"]
    
    # Định nghĩa ma trận sự tham gia của các tác tử trong mỗi giai đoạn
    # 1: Tác tử tham gia vào giai đoạn, 0: Không tham gia
    participation = np.array([
        [1, 0, 0, 0, 0, 0],  # Reader
        [0, 1, 1, 1, 1, 1],  # Planner
        [0, 1, 1, 1, 1, 1],  # Developer
        [0, 1, 1, 1, 1, 1],  # Reviewer
        [0, 1, 1, 1, 1, 1]   # Summarizer
    ])
    
    # Định nghĩa màu sắc cho mỗi tác tử
    agent_colors = ['#FFC107', '#4CAF50', '#2196F3', '#9C27B0', '#FF5722']
    
    # Vẽ các giai đoạn
    phase_boxes = []
    for i, phase in enumerate(phases):
        box = mpatches.Rectangle((i, -1), 0.8, 7, 
                              fill=True, alpha=0.1, 
                              edgecolor='gray', facecolor='lightgray')
        ax.add_patch(box)
        phase_boxes.append(box)
        ax.text(i + 0.4, -0.5, phase, ha='center', va='center', fontsize=10, fontweight='bold')
    
    # Vẽ các tác tử và hoạt động của họ
    for i, agent in enumerate(agents):
        ax.text(-0.5, i + 0.5, agent, ha='right', va='center', fontsize=12, fontweight='bold')
        
        for j in range(len(phases)):
            if participation[i, j] == 1:
                circle = mpatches.Circle((j + 0.4, i + 0.5), 0.3, 
                                      fill=True, alpha=0.7,
                                      edgecolor='black', facecolor=agent_colors[i])
                ax.add_patch(circle)
                ax.text(j + 0.4, i + 0.5, agent[0], ha='center', va='center', fontweight='bold', color='white')
    
    # Vẽ mũi tên kết nối giữa các giai đoạn
    for i in range(len(phases) - 1):
        arrow = mpatches.FancyArrowPatch((i + 0.8, -0.8), (i + 1, -0.8), 
                                       connectionstyle="arc3,rad=0", 
                                       arrowstyle='-|>', 
                                       mutation_scale=20, 
                                       lw=2, 
                                       color='black')
        ax.add_patch(arrow)
    
    # Tạo chú thích
    legend_patches = []
    for i, agent in enumerate(agents):
        legend_patches.append(mpatches.Patch(color=agent_colors[i], label=agent))
    
    ax.legend(handles=legend_patches, loc='upper center', bbox_to_anchor=(0.5, 1.1), ncol=5)
    
    # Thiết lập trục
    ax.set_xlim(-1, len(phases))
    ax.set_ylim(-1.5, len(agents))
    ax.set_title('AutoKaggle: Phase-Based Multi-Agent System', fontsize=16)
    
    # Tắt trục
    ax.axis('off')
    
    plt.tight_layout()
    
    return fig

# Vẽ sơ đồ (comment lại để không thực thi)
# fig = visualize_multi_agent_system()
# plt.show()

## Kết Luận

Hệ thống đa tác tử dựa trên giai đoạn trong AutoKaggle là một cách tiếp cận mạnh mẽ để tự động hóa các quy trình khoa học dữ liệu phức tạp. Bằng cách phân chia quy trình thành các giai đoạn rõ ràng và sử dụng các tác tử chuyên biệt, AutoKaggle có thể giải quyết các vấn đề phức tạp một cách có tổ chức và hiệu quả.

### Những Điểm Chính Cần Nhớ:

1. **Phân Chia Quy Trình**: Chia nhỏ quy trình khoa học dữ liệu thành 6 giai đoạn riêng biệt giúp quản lý nhiệm vụ phức tạp

2. **Chuyên Biệt Hóa Tác Tử**: Mỗi tác tử có một vai trò cụ thể, từ đọc thông tin (Reader) đến lập kế hoạch (Planner), viết code (Developer), đánh giá (Reviewer) và tổng kết (Summarizer)

3. **Phối Hợp Dựa Trên State**: Sử dụng cơ chế trạng thái chung để truyền thông tin giữa các tác tử và giai đoạn

4. **Quy Trình Lặp**: Cho phép các lỗi được phát hiện và sửa chữa trước khi chuyển sang giai đoạn tiếp theo

5. **Tính Mở Rộng**: Thiết kế module hóa cho phép dễ dàng thêm tác tử mới hoặc điều chỉnh luồng công việc

Bằng cách áp dụng thiết kế tương tự, các nhà phát triển có thể xây dựng hệ thống tự động cho nhiều lĩnh vực khác ngoài khoa học dữ liệu, đặc biệt là những lĩnh vực có quy trình phức tạp cần nhiều bước phối hợp.